In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt

# Load the dataset
url = 'https://raw.githubusercontent.com/noobstang/NNtraining/master/Weather49Sets/weatherstats_ottawa_daily.csv'
data = pd.read_csv(url)
data['date'] = pd.to_datetime(data['date'])
data_filtered = data[(data['date'].dt.year >= 2010) & (data['date'].dt.year <= 2023)]

# Select the required columns and preprocess
columns_required = ['avg_hourly_temperature', 'precipitation', 'avg_hourly_pressure_station', 'solar_radiation']
data_filtered = data_filtered[columns_required].dropna()

# Scale the data
scaler = MinMaxScaler(feature_range=(0, 1))
data_scaled = scaler.fit_transform(data_filtered)

def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
    # Input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # Forecast sequence (t, t+1, ... t+n_out)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # Put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # Drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

# Convert to supervised learning problem with a window size of 3
data_supervised = series_to_supervised(data_scaled, 7, 1)

# Split the dataset
n_obs = 7 * 4  # Update for 3-day window size
values = data_supervised.values
n_train_days = int(len(values) * 0.8)
train = values[:n_train_days, :]
test = values[n_train_days:, :]
train_X, train_y = train[:, :n_obs], train[:, -4:]
test_X, test_y = test[:, :n_obs], test[:, -4:]

# Convert to tensors
train_X = torch.tensor(train_X, dtype=torch.float32)
train_y = torch.tensor(train_y, dtype=torch.float32)
test_X = torch.tensor(test_X, dtype=torch.float32)
test_y = torch.tensor(test_y, dtype=torch.float32)

# DataLoader
train_dataset = TensorDataset(train_X, train_y)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_dataset = TensorDataset(test_X, test_y)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Define MLP Model with adjusted input dimension
class MLP(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(MLP, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, output_dim)
        )

    def forward(self, x):
        return self.network(x)

# Initialize the model with the updated input dimension
model = MLP(28, 4)  # Update for 3-day window size
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
epochs = 50
for epoch in range(epochs):
    model.train()
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        output = model(inputs)
        loss = criterion(output, targets)
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch+1}/{epochs}, Loss: {loss.item()}')



# Evaluate the model
def get_predictions_and_targets(loader, model):
    model.eval()  # Set the model to evaluation mode
    predictions, targets = [], []
    with torch.no_grad():
        for inputs, labels in loader:
            outputs = model(inputs)
            predictions.append(outputs.numpy())
            targets.append(labels.numpy())
    predictions = np.vstack(predictions)
    targets = np.vstack(targets)
    return predictions, targets


model.eval()
with torch.no_grad():
    predictions = model(test_X)
    test_loss = criterion(predictions, test_y)
print(f'Test Loss: {test_loss.item()}')

# MAE and MSE calculation (already in your code)
from sklearn.metrics import mean_absolute_error, mean_squared_error
train_predictions, train_targets = get_predictions_and_targets(train_loader, model)
test_predictions, test_targets = get_predictions_and_targets(test_loader, model)
train_mae = mean_absolute_error(train_targets, train_predictions)
train_mse = mean_squared_error(train_targets, train_predictions)
test_mae = mean_absolute_error(test_targets, test_predictions)
test_mse = mean_squared_error(test_targets, test_predictions)
print(f"Training MAE: {train_mae}, Training MSE: {train_mse}")
print(f"Testing MAE: {test_mae}, Testing MSE: {test_mse}")


<ipython-input-5-c436d6738c67>:12: DtypeWarning: Columns (46,49) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(url)


Epoch 1/50, Loss: 0.020805135369300842
Epoch 2/50, Loss: 0.019271599128842354
Epoch 3/50, Loss: 0.019651232287287712
Epoch 4/50, Loss: 0.013988095335662365
Epoch 5/50, Loss: 0.013530207797884941
Epoch 6/50, Loss: 0.012014723382890224
Epoch 7/50, Loss: 0.012518925592303276
Epoch 8/50, Loss: 0.012947366572916508
Epoch 9/50, Loss: 0.011041009798645973
Epoch 10/50, Loss: 0.010950897820293903
Epoch 11/50, Loss: 0.010867259465157986
Epoch 12/50, Loss: 0.011627870611846447
Epoch 13/50, Loss: 0.012205924838781357
Epoch 14/50, Loss: 0.012859693728387356
Epoch 15/50, Loss: 0.012238586321473122
Epoch 16/50, Loss: 0.012560316361486912
Epoch 17/50, Loss: 0.00891274306923151
Epoch 18/50, Loss: 0.01020968146622181
Epoch 19/50, Loss: 0.011531354859471321
Epoch 20/50, Loss: 0.010908873751759529
Epoch 21/50, Loss: 0.011988423764705658
Epoch 22/50, Loss: 0.012605826370418072
Epoch 23/50, Loss: 0.013834776356816292
Epoch 24/50, Loss: 0.011762264184653759
Epoch 25/50, Loss: 0.01432870328426361
Epoch 26/50,



```
3 days window:
Test Loss: 0.010468280874192715
Training MAE: 0.07376331835985184, Training MSE: 0.010967354290187359
Testing MAE: 0.07252750545740128, Testing MSE: 0.01046828180551529
```

